In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

from PIL import Image as PILImage

import cv2
import numpy as np
import glob
import os

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import Sequence

from keras.layers import Layer

import csv
import pandas as pd

In [2]:
IMAGE_SIZE = (256, 256)

In [3]:
def imshow(a, size=1.0):
    # Clip and convert the image to uint8
    a = a.clip(0, 255).astype("uint8")
    
    # Resize the image if a size factor is provided
    if size != 1.0:
        new_dim = (int(a.shape[1] * size), int(a.shape[0] * size))
        a = cv2.resize(a, new_dim, interpolation=cv2.INTER_AREA)
    
    # Display the image
    display(PILImage.fromarray(a))

In [4]:
def resize_crop(image, image_size=(256, 256)):
    target_height, target_width = image_size
    original_height, original_width = image.shape[:2]

    original_aspect = original_width / original_height
    target_aspect = target_width / target_height

    if original_aspect > target_aspect:
        new_width = original_height
        crop_x = (original_width - new_width) // 2
        cropped_image = image[:, crop_x:crop_x + new_width]
    elif original_aspect < target_aspect:
        new_height = original_width
        crop_y = (original_height - new_height) // 2
        cropped_image = image[crop_y:crop_y + new_height, :]
    else:
        cropped_image = image

    resized_image = cv2.resize(cropped_image, (target_width, target_height))
    return resized_image

def preprocess(image):
    image = image / 255.0
    image = resize_crop(image)
    image = cv2.GaussianBlur(image, (5, 5), 0)
    return image

def read_image(image_file):
    image = cv2.imread(image_file)
    image = preprocess(image) 
    return image 

In [5]:
data_folder = "Data_Final/*"
image_files = glob.glob(os.path.join(data_folder, "*.jpg"), recursive=True)

In [ ]:
print(image_files[0])

In [ ]:
parsed = image_files[0].replace("Data_Final/", "")
print(parsed)

In [8]:
class L2Normalization(Layer):
    def call(self, inputs):
        return tf.math.l2_normalize(inputs, axis=1)

In [9]:
# model = load_model("Models/transfer_custom_big_2.keras",
#     custom_objects={"L2Normalization": L2Normalization})

In [ ]:
model = load_model("Models/ResNet50.keras")

In [ ]:
def extract_and_save_embeddings(image_paths, embedding_model, save_path='embeddings.csv'):
    embeddings = []
    filenames = []
    
    count = 0
    for image_path in image_paths:
        image = read_image(image_path)
        file_name = image_path.replace("Data_Final/", "")
        
        image = np.expand_dims(image, axis=0)  
        
        embedding = embedding_model.predict(image, verbose=0)
        
        embeddings.append(embedding.flatten())
        filenames.append(file_name)  
        
        if count %300 == 0:
            print(f"Processed {count}/{len(image_paths)} images")
        count += 1
    
    embeddings = np.array(embeddings)
    df = pd.DataFrame(embeddings)
    df['filename'] = filenames
    df.to_csv(save_path, index=False)
    print(f"Embeddings saved to {save_path}")


extract_and_save_embeddings(image_files , model, save_path='CSVs/ResNet50.csv')